In [93]:
from __future__ import division
from __future__ import print_function

import numpy as np
import matplotlib

matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

# Generating data

In [94]:
np.random.seed(0)  # set the random seed for reproducibility

def generate_sample(xmin, xmax, sample_size):
    x = np.linspace(start=xmin, stop=xmax, num=sample_size)
    pix = np.pi * x
    target = np.sin(pix) / pix + 0.1 * x
    noise = 0.05 * np.random.normal(loc=0., scale=1., size=sample_size)
    return x, target, target + noise


def calc_design_matrix(x, c, h):
    return np.exp(-(x[None] - c[:, None]) ** 2 / (2 * h ** 2))

In [95]:
# create sample
sample_size = 50
xmin, xmax = -3, 3
x, ytrue, y = generate_sample(xmin=xmin, xmax=xmax, sample_size=sample_size)

# Cross Validation

In [96]:
def cross_validation(x,y,lm,hh,sample_size):
    # Erros list
    errors = np.zeros((hh.size,lm.size,))
    # Theta list
    thetas = []

    # loop over the badwidths h
    for n1,h in enumerate(hh):
        # loop over the lambda values l
        for n2,l in enumerate(lm):
            # loop over the dataset leaving out one sample at each iteration
            err = 0 # Store test errors
            for i in range(sample_size):

                # Compute cross validation training and validation data
                x_val = np.atleast_1d(x[i])
                y_val = y[i]
                x_loocv = np.delete(x,i)
                y_loocv = np.delete(y,i)
                # calculate design matrix
                k = calc_design_matrix(x_loocv, x_loocv, h)
                # Solve the least square problem
                theta = np.linalg.solve(
                    k.T.dot(k) + l * np.identity(len(k)),
                    k.T.dot(y_loocv[:, None]))
                # Compute prediction 
                K = calc_design_matrix(x_loocv, x_val, h)
                prediction = K.dot(theta)
                # Compute squared error and store
                err+=(np.ndarray.item(prediction)-y[i])**2
            # Store mean errors for different parameter values.
            errors[n1,n2]=(err/sample_size)
            # Store the learned parameter
            thetas.append(theta)
    
    min_in = np.unravel_index(errors.argmin(), errors.shape)
    print(f"Minimum Cross Validation Error is {errors[min_in]} at Lambda = {lm[min_in[1]]} and Bandwidth = {hh[min_in[0]]}")

    return thetas,errors
            

# Run Cross Validation

In [97]:
#define lambda values
lm = np.array([0.0001,0.1,100])
#define range of gaussian bandwidth h
hh = np.array([0.03,0.3,3])

thetas,errors = cross_validation(x,y,lm,hh,sample_size)


Minimum Cross Validation Error is 0.0037134179245161066 at Lambda = 0.1 and Bandwidth = 0.3


# Plotting

In [98]:

# create data to visualize the prediction
X = np.linspace(start=xmin, stop=xmax, num=5000)

# define subplot grid
fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(20, 20))
plt.subplots_adjust(hspace=0.5)
fig.suptitle("Model selection w/ regularized regression", fontsize=18, y=0.95)

comb_array = np.array(np.meshgrid(hh, lm)).T.reshape(-1, 2)


In [ ]:

for theta,c,ax in zip(thetas,comb_array,axs.ravel()):
    #print(theta,l,h)
    h,l = c[0],c[1]
    K = calc_design_matrix(x[:-1], X, h)
    prediction = K.dot(theta)
    # visualization
    ax.scatter(x, y, c='blue', marker='o')
    ax.plot(x, ytrue, c='red',label="Ground Truth")
    ax.plot(X, prediction, c="green", label="Prediction")
    
    plt.legend()
    fig.supxlabel('lambda = [0.0001,0.1,100]')
    fig.supylabel('h = [0.03,0.3,3]')

plt.show()
#plt.savefig('output.png')